# Proses Gambar Sumber RAW Output A

In [1]:
import os
import csv
from tqdm import tqdm  # pastikan sudah install: pip install tqdm

import importlib
import lib.export
import lib.A_acc
importlib.reload(lib.export)
importlib.reload(lib.A_acc)
from lib.export import ekspor_png
from lib.A_acc import get_color_corrected_increased_dynamic_range

def batch_lab_metrics_resume(
    input_folder: str,
    process_name: str,
    output_csv: str,
    cie: bool = True,
    device: str = "cuda",
    exts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"),
    recursive: bool = False,
    limit: int | None = None,
):
    """
    Batch processing dengan resume otomatis dari CSV + progress bar.
    - Baca CSV (jika ada) → skip file yang sudah diproses.
    - Proses hanya file sisa (bisa dibatasi 'limit').
    - Progress bar ditampilkan dengan tqdm.
    - Append hasil ke CSV (header ditulis kalau CSV belum ada).
    """
    # 1) Kumpulkan semua file gambar
    all_files = []
    walker = os.walk(input_folder) if recursive else [(input_folder, [], os.listdir(input_folder))]
    for root, _, files in walker:
        for fname in files:
            if fname.lower().endswith(exts):
                all_files.append(os.path.abspath(os.path.join(root, fname)))
    all_files.sort()

    # 2) Baca CSV untuk skip file yang sudah diproses
    done = set()
    # if os.path.exists(output_csv):
    #     try:
    #         with open(output_csv, "r", newline="", encoding="utf-8") as f:
    #             reader = csv.reader(f)
    #             header = next(reader, None)
    #             for row in reader:
    #                 if row:
    #                     done.add(os.path.abspath(row[0]))
    #     except Exception as e:
    #         print(f"[PERINGATAN] Gagal baca CSV: {e}, lanjut tanpa resume.")
    #         done = set()

    # 3) Tentukan sisa file
    remaining = [p for p in all_files if p not in done]
    if limit is not None:
        remaining = remaining[:max(0, limit)]

    if not remaining:
        print(f"Tidak ada file baru untuk diproses. Total sudah diproses: {len(done)}.")
        return 0

    # 4) Proses file dengan progress bar
    rows = []
    iter = 0
    for fpath in tqdm(remaining, desc="Processing images", unit="img"):
        filename = os.path.basename(fpath)

        im_increased, cast = get_color_corrected_increased_dynamic_range(fpath, device="cuda")
        ekspor_png(im_increased, "../UIEB/results/" + process_name, filename)


        rows.append([
            iter,
            filename,
            cast,
        ])
        iter = iter + 1

    # 5) Tulis ke CSV
    os.makedirs(os.path.dirname(output_csv) or ".", exist_ok=True)
    write_header = not os.path.exists(output_csv)
    with open(output_csv, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(["id","filename", "cast"])
        writer.writerows(rows)

    print(f"Selesai. Diproses {len(rows)} file baru. Total sebelumnya: {len(done)}. "
          f"Total sekarang (CSV) ≈ {len(done) + len(rows)}.")
    return len(rows)


In [2]:
source_folder = "../UIEB/raw-890"
out_folder = ""
process_name = "2024_haofeng_hu_a" #jadi folder dan nama file csv
batch_lab_metrics_resume(source_folder, process_name, "hasil/"+process_name+".csv", limit=890)

Processing images: 100%|██████████| 890/890 [01:36<00:00,  9.25img/s]

Selesai. Diproses 890 file baru. Total sebelumnya: 0. Total sekarang (CSV) ≈ 890.


890